In [65]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# =========================
# 1. Wczytanie danych
# =========================
df = pd.read_csv("european_capitals_history_clean2.csv", parse_dates=["time"])

# =========================
# 2. Lista miast
# =========================
cities = [
    "Warszawa","Berlin","Praga","Budapeszt","Madryt","Paryż","Londyn","Wiedeń","Bruksela",
    "Amsterdam","Lizbona","Oslo","Sztokholm","Helsinki","Dublin","Ateny","Belgrad","Sofia",
    "Tallinn","Ryga","Wilno","Luxemburg","Vaduz","Monako","San Marino","Kiszyniów","Sarajewo",
    "Podgorica","Skopje","Mińsk","Ljubljana","Zagrzeb","Kijów","Moskwa","Reykjavik"
]

features = ['temp','tmin','tmax','wspd','pres','temp_rolling3','temp_diff1']

WINDOW = 30
HORIZON = 5

os.makedirs("model/lstm", exist_ok=True)

for city in cities:
    print(f"\n🔹 Tworzenie modelu dla miasta: {city}")

    # =========================
    # 3. Wybranie danych dla miasta
    # =========================
    df_city = df[df['city'] == city][['time'] + features].copy()
    df_city.sort_values('time', inplace=True)
    df_city.reset_index(drop=True, inplace=True)

    if len(df_city) < WINDOW + HORIZON:
        print(f"⚠️ Za mało danych dla {city}, pomijam.")
        continue

    # =========================
    # 4. Funkcja do sekwencji
    # =========================
    def create_sequences(df, features, window=WINDOW, horizon=HORIZON):
        X, y = [], []
        for i in range(len(df) - window - horizon + 1):
            X.append(df.iloc[i:i+window][features].values)
            y.append(df.iloc[i+window:i+window+horizon]['temp'].values)
        return np.array(X), np.array(y)

    X, y = create_sequences(df_city, features)

    # =========================
    # 5. Train/Test split
    # =========================
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # =========================
    # 6. Skalowanie
    # =========================
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_reshaped = X_train.reshape(-1, len(features))
    X_test_reshaped  = X_test.reshape(-1, len(features))

    scaler_X.fit(X_train_reshaped)
    scaler_y.fit(y_train)

    X_train_scaled = scaler_X.transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled  = scaler_X.transform(X_test_reshaped).reshape(X_test.shape)

    y_train_scaled = scaler_y.transform(y_train)
    y_test_scaled  = scaler_y.transform(y_test)

    # =========================
    # 7. Model LSTM
    # =========================
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(WINDOW, len(features))),
        Dropout(0.2),
        LSTM(32),
        Dense(HORIZON)
    ])

    model.compile(optimizer='adam', loss='mse')

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # =========================
    # 8. Trening
    # =========================
    model.fit(
        X_train_scaled, y_train_scaled,
        epochs=100,
        batch_size=32,
        validation_data=(X_test_scaled, y_test_scaled),
        callbacks=[early_stop],
        verbose=0
    )

    # =========================
    # 9. Zapis modelu
    # =========================
    model_file = f"model/lstm/{city.lower().replace(' ','_')}_lstm_multivariate_{HORIZON}dni.keras"
    model.save(model_file)
    print(f"✅ Model dla {city} zapisany jako {model_file}")



🔹 Tworzenie modelu dla miasta: Warszawa


C:\Users\gduls\PycharmProjects\pythonProject10\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model dla Warszawa zapisany jako model/lstm/warszawa_lstm_multivariate_5dni.keras

🔹 Tworzenie modelu dla miasta: Berlin


C:\Users\gduls\PycharmProjects\pythonProject10\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model dla Berlin zapisany jako model/lstm/berlin_lstm_multivariate_5dni.keras

🔹 Tworzenie modelu dla miasta: Praga


...

✅ Model dla Kijów zapisany jako model/lstm/kijów_lstm_multivariate_5dni.keras

🔹 Tworzenie modelu dla miasta: Moskwa


C:\Users\gduls\PycharmProjects\pythonProject10\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model dla Moskwa zapisany jako model/lstm/moskwa_lstm_multivariate_5dni.keras

🔹 Tworzenie modelu dla miasta: Reykjavik


C:\Users\gduls\PycharmProjects\pythonProject10\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model dla Reykjavik zapisany jako model/lstm/reykjavik_lstm_multivariate_5dni.keras


In [66]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os

# =========================
# PARAMETRY
# =========================
WINDOW = 30
HORIZON = 5
features = ['temp','tmin','tmax','wspd','pres','temp_rolling3','temp_diff1']

# =========================
# 1. Lista miast
# =========================
cities = [
    "Warszawa","Berlin","Praga","Budapeszt","Madryt","Paryż","Londyn","Wiedeń","Bruksela",
    "Amsterdam","Lizbona","Oslo","Sztokholm","Helsinki","Dublin","Ateny","Belgrad","Sofia",
    "Tallinn","Ryga","Wilno","Luxemburg","Vaduz","Monako","San Marino","Kiszyniów","Sarajewo",
    "Podgorica","Skopje","Mińsk","Ljubljana","Zagrzeb","Kijów","Moskwa","Reykjavik"
]

# =========================
# 2. Wczytanie danych historycznych
# =========================
df = pd.read_csv("european_capitals_history_clean2.csv", parse_dates=["time"])

# =========================
# 3. Listy do prognoz i braków
# =========================
all_forecasts = []
missing_data_cities = []

# =========================
# 4. Prognoza dla każdego miasta
# =========================
for city in cities:
    model_file = f"model/lstm/{city.lower().replace(' ','_')}_lstm_multivariate_{HORIZON}dni.keras"
    
    missing_reasons = []

    if not os.path.exists(model_file):
        missing_reasons.append("brak modelu")

    df_city = df[df['city'] == city][['time'] + features].copy()
    df_city.sort_values('time', inplace=True)
    df_city.reset_index(drop=True, inplace=True)

    if len(df_city) < WINDOW + HORIZON:
        missing_reasons.append(f"za mało danych (wymagane min {WINDOW + HORIZON}, jest {len(df_city)})")

    missing_cols = df_city[features].columns[df_city[features].isnull().any()].tolist()
    if missing_cols:
        missing_reasons.append(f"brakujące kolumny: {', '.join(missing_cols)}")

    if missing_reasons:
        print(f"\n⚠️ Nie można wygenerować prognozy dla {city}: { '; '.join(missing_reasons) }")
        missing_data_cities.append({"city": city, "reasons": missing_reasons})
        continue

    print(f"\n🔹 Prognoza dla miasta: {city}")

    # =========================
    # Wczytanie modelu
    # =========================
    model = load_model(model_file)

    # =========================
    # Skalowanie danych
    # =========================
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_data = df_city[features].iloc[:-HORIZON].values
    y_data = df_city[['temp']].iloc[WINDOW:].values

    scaler_X.fit(X_data)
    scaler_y.fit(y_data)

    # =========================
    # Ostatnia sekwencja
    # =========================
    last_sequence = df_city[features].iloc[-WINDOW:].values
    last_sequence_scaled = scaler_X.transform(last_sequence).reshape(1, WINDOW, len(features))

    # =========================
    # Prognoza
    # =========================
    future_scaled = model.predict(last_sequence_scaled, verbose=0)
    future_temp = scaler_y.inverse_transform(future_scaled).flatten()

    # =========================
    # Daty prognozy
    # =========================
    last_date = df_city['time'].iloc[-1]
    future_dates = pd.date_range(
        start=last_date + pd.Timedelta(days=1),
        periods=HORIZON,
        freq='D'
    )

    # =========================
    # Dodanie prognoz do listy
    # =========================
    for date, temp in zip(future_dates, future_temp):
        all_forecasts.append({
            "city": city,
            "date": date.date(),
            "temp_forecast": round(temp, 2)
        })

# =========================
# 5. Zapis prognoz do CSV
# =========================
df_forecast = pd.DataFrame(all_forecasts)
df_forecast.to_csv("forecasts/forecasts_5dni_all_cities_lstm.csv", index=False)
print("\n✅ Wszystkie prognozy zapisane w forecasts_5dni_all_cities_lstm.csv")

# =========================
# 6. Wyświetlenie miast z brakami danych
# =========================
if missing_data_cities:
    print("\n❌ Miasta, dla których nie udało się wygenerować prognozy:")
    for item in missing_data_cities:
        print(f"- {item['city']}: {', '.join(item['reasons'])}")



🔹 Prognoza dla miasta: Warszawa

🔹 Prognoza dla miasta: Berlin

🔹 Prognoza dla miasta: Praga

🔹 Prognoza dla miasta: Budapeszt

🔹 Prognoza dla miasta: Madryt

🔹 Prognoza dla miasta: Paryż

🔹 Prognoza dla miasta: Londyn

🔹 Prognoza dla miasta: Wiedeń

🔹 Prognoza dla miasta: Bruksela

🔹 Prognoza dla miasta: Amsterdam

🔹 Prognoza dla miasta: Lizbona

🔹 Prognoza dla miasta: Oslo

🔹 Prognoza dla miasta: Sztokholm

🔹 Prognoza dla miasta: Helsinki

🔹 Prognoza dla miasta: Dublin

🔹 Prognoza dla miasta: Ateny

🔹 Prognoza dla miasta: Belgrad

🔹 Prognoza dla miasta: Sofia

🔹 Prognoza dla miasta: Tallinn

🔹 Prognoza dla miasta: Ryga

🔹 Prognoza dla miasta: Wilno

🔹 Prognoza dla miasta: Luxemburg

🔹 Prognoza dla miasta: Vaduz

🔹 Prognoza dla miasta: Monako

🔹 Prognoza dla miasta: San Marino

🔹 Prognoza dla miasta: Kiszyniów

🔹 Prognoza dla miasta: Sarajewo

🔹 Prognoza dla miasta: Podgorica

🔹 Prognoza dla miasta: Skopje

🔹 Prognoza dla miasta: Mińsk

🔹 Prognoza dla miasta: Ljubljana

🔹 Prognoza dl

In [46]:
import os
import pandas as pd
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

# =========================
# 1. Wczytanie danych
# =========================
df = pd.read_csv("european_capitals_history_clean2.csv", parse_dates=["time"])

# =========================
# 2. Lista miast
# =========================
cities = [
    "Warszawa","Berlin","Praga","Budapeszt","Madryt","Paryż","Londyn","Wiedeń","Bruksela",
    "Amsterdam","Lizbona","Oslo","Sztokholm","Helsinki","Dublin","Ateny","Belgrad","Sofia",
    "Tallinn","Ryga","Wilno","Luxemburg","Vaduz","Monako","San Marino","Kiszyniów","Sarajewo",
    "Podgorica","Skopje","Mińsk","Ljubljana","Zagrzeb","Kijów","Moskwa","Reykjavik"
]

# Kolumny używane jako regresory
regressors = ['tmin','tmax','wspd','pres','temp_rolling3','temp_diff1']

# Folder do zapisania modeli
os.makedirs("models/prophet", exist_ok=True)

# =========================
# 3. Pętla po miastach
# =========================
for city in cities:
    print(f"\n🔹 Tworzenie modelu Prophet dla miasta: {city}")
    
    df_city = df[df['city'] == city][['time','temp'] + regressors].copy()
    df_city.sort_values('time', inplace=True)
    df_city.reset_index(drop=True, inplace=True)
    
    if len(df_city) < 365:  # minimum 1 rok danych
        print(f"⚠️ Za mało danych dla {city}, pomijam.")
        continue
    
    # Prophet wymaga kolumn 'ds' i 'y'
    df_city.rename(columns={'time':'ds','temp':'y'}, inplace=True)
    
    # =========================
    # 4. Definicja modelu Prophet z regresorami
    # =========================
    model = Prophet(
        yearly_seasonality=15,
        weekly_seasonality=10,
        daily_seasonality=False,
        changepoint_prior_scale=0.1
    )
    
    # Sezonowość miesięczna
    model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Dodanie regresorów
    for reg in regressors:
        model.add_regressor(reg)
    
    # =========================
    # 5. Trenowanie modelu
    # =========================
    model.fit(df_city)
    
    # =========================
    # 6. Zapis modelu do pliku JSON
    # =========================
    model_file = f"model/prophet/{city.lower().replace(' ','_')}_prophet.json"
    with open(model_file, 'w') as f:
        f.write(model_to_json(model))
    print(f"✅ Model Prophet dla {city} zapisany jako {model_file}")



🔹 Tworzenie modelu Prophet dla miasta: Warszawa


17:22:58 - cmdstanpy - INFO - Chain [1] start processing
17:22:59 - cmdstanpy - INFO - Chain [1] done processing


✅ Model Prophet dla Warszawa zapisany jako model/prophet/warszawa_prophet.json

🔹 Tworzenie modelu Prophet dla miasta: Berlin


17:22:59 - cmdstanpy - INFO - Chain [1] start processing
17:22:59 - cmdstanpy - INFO - Chain [1] done processing


✅ Model Prophet dla Berlin zapisany jako model/prophet/berlin_prophet.json

🔹 Tworzenie modelu Prophet dla miasta: Praga


...

✅ Model Prophet dla Kijów zapisany jako model/prophet/kijów_prophet.json

🔹 Tworzenie modelu Prophet dla miasta: Moskwa


17:23:22 - cmdstanpy - INFO - Chain [1] start processing
17:23:22 - cmdstanpy - INFO - Chain [1] done processing


✅ Model Prophet dla Moskwa zapisany jako model/prophet/moskwa_prophet.json

🔹 Tworzenie modelu Prophet dla miasta: Reykjavik


17:23:23 - cmdstanpy - INFO - Chain [1] start processing
17:23:23 - cmdstanpy - INFO - Chain [1] done processing


✅ Model Prophet dla Reykjavik zapisany jako model/prophet/reykjavik_prophet.json


In [53]:
import os
import pandas as pd
from prophet.serialize import model_from_json
from prophet import Prophet

# =========================
# PARAMETRY
# =========================
HORIZON = 5  # prognoza na 5 dni
regressors = ['tmin','tmax','wspd','pres','temp_rolling3','temp_diff1']

# =========================
# Lista miast
# =========================
cities = [
    "Warszawa","Berlin","Praga","Budapeszt","Madryt","Paryż","Londyn","Wiedeń","Bruksela",
    "Amsterdam","Lizbona","Oslo","Sztokholm","Helsinki","Dublin","Ateny","Belgrad","Sofia",
    "Tallinn","Ryga","Wilno","Luxemburg","Vaduz","Monako","San Marino","Kiszyniów","Sarajewo",
    "Podgorica","Skopje","Mińsk","Ljubljana","Zagrzeb","Kijów","Moskwa","Reykjavik"
]

# =========================
# Wczytanie danych historycznych
# =========================
df = pd.read_csv("european_capitals_history_clean2.csv", parse_dates=["time"])
all_forecasts = []
missing_data_cities = []

# =========================
# Prognoza dla każdego miasta
# =========================
for city in cities:
    model_file = f"model/prophet/{city.lower().replace(' ','_')}_prophet.json"
    missing_reasons = []

    # Sprawdzenie, czy model istnieje
    if not os.path.exists(model_file):
        missing_reasons.append("brak modelu")
    
    # Dane dla miasta
    df_city = df[df['city'] == city][['time','temp'] + regressors].copy()
    df_city.sort_values('time', inplace=True)
    df_city.reset_index(drop=True, inplace=True)
    df_city.rename(columns={'time':'ds','temp':'y'}, inplace=True)
    
    # Sprawdzenie długości danych
    if len(df_city) < 365:
        missing_reasons.append("za mało danych (<1 rok)")
    
    # Sprawdzenie brakujących kolumn
    missing_cols = df_city.columns[df_city.isnull().any()].tolist()
    if missing_cols:
        missing_reasons.append(f"brakujące kolumny: {', '.join(missing_cols)}")
    
    if missing_reasons:
        print(f"\n⚠️ Nie można wygenerować prognozy dla {city}: {'; '.join(missing_reasons)}")
        missing_data_cities.append({"city": city, "reasons": missing_reasons})
        continue
    
    print(f"\n🔹 Prognoza Prophet dla miasta: {city}")
    
    # =========================
    # Wczytanie modelu
    # =========================
    with open(model_file, 'r') as f:
        model = model_from_json(f.read())
    
    # =========================
    # Prognoza na następne 5 dni
    # =========================
    future = model.make_future_dataframe(periods=HORIZON)

    # 🔹 Dodanie regresorów dla przyszłości
    for reg in regressors:
        # Weź ostatnie HORIZON wartości z historycznych danych
        last_values = df_city[reg].iloc[-HORIZON:].values
        if len(last_values) < HORIZON:
            # jeśli mniej niż HORIZON dni danych, powtarzamy ostatnią wartość
            last_values = [df_city[reg].iloc[-1]] * HORIZON
        # Rozszerzamy kolumnę regresora
        future[reg] = pd.concat([df_city[reg], pd.Series(last_values)]).reset_index(drop=True).iloc[-len(future):]

    # Generowanie prognozy
    forecast = model.predict(future)
    
    # =========================
    # Zapis prognozy 5 dni do listy
    # =========================
    forecast_5days = forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(HORIZON)
    for idx, row in forecast_5days.iterrows():
        all_forecasts.append({
            "city": city,
            "date": row['ds'].date(),
            "temp_forecast": round(row['yhat'],2),
            "temp_lower": round(row['yhat_lower'],2),
            "temp_upper": round(row['yhat_upper'],2)
        })

# =========================
# Zapis prognoz do CSV
# =========================
df_forecast = pd.DataFrame(all_forecasts)
os.makedirs("forecasts", exist_ok=True)
df_forecast.to_csv("forecasts/forecasts_5dni_all_cities_prophet.csv", index=False)
print("\n✅ Wszystkie prognozy zapisane w forecasts/forecasts_5dni_all_cities_prophet.csv")

# =========================
# Wyświetlenie miast z brakami danych
# =========================
if missing_data_cities:
    print("\n❌ Miasta, dla których nie udało się wygenerować prognozy:")
    for item in missing_data_cities:
        print(f"- {item['city']}: {', '.join(item['reasons'])}")



🔹 Prognoza Prophet dla miasta: Warszawa

🔹 Prognoza Prophet dla miasta: Berlin

🔹 Prognoza Prophet dla miasta: Praga

🔹 Prognoza Prophet dla miasta: Budapeszt

🔹 Prognoza Prophet dla miasta: Madryt

🔹 Prognoza Prophet dla miasta: Paryż

🔹 Prognoza Prophet dla miasta: Londyn

🔹 Prognoza Prophet dla miasta: Wiedeń

🔹 Prognoza Prophet dla miasta: Bruksela

🔹 Prognoza Prophet dla miasta: Amsterdam

🔹 Prognoza Prophet dla miasta: Lizbona

🔹 Prognoza Prophet dla miasta: Oslo

🔹 Prognoza Prophet dla miasta: Sztokholm

🔹 Prognoza Prophet dla miasta: Helsinki

🔹 Prognoza Prophet dla miasta: Dublin

🔹 Prognoza Prophet dla miasta: Ateny

🔹 Prognoza Prophet dla miasta: Belgrad

🔹 Prognoza Prophet dla miasta: Sofia

🔹 Prognoza Prophet dla miasta: Tallinn

🔹 Prognoza Prophet dla miasta: Ryga

🔹 Prognoza Prophet dla miasta: Wilno

🔹 Prognoza Prophet dla miasta: Luxemburg

🔹 Prognoza Prophet dla miasta: Vaduz

🔹 Prognoza Prophet dla miasta: Monako

🔹 Prognoza Prophet dla miasta: San Marino

🔹 Prognoz